#### Reference DF and DF per item purchased

In [8]:
import re
import json
import pandas as pd
import numpy as np
import calendar
with open("transaction-data-adhoc-analysis.json","r") as f:
    tdata = json.load(f)

#Original DataFrame - do not touch will mainly be used for referencing
df = pd.DataFrame(tdata)

#split rows based on this: https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows
lst_col = "transaction_items" #Name in the .json file
x = df.assign(**{lst_col:df[lst_col].str.split(';')}) 
df_perItem = pd.DataFrame(
    {col:np.repeat(x[col].values, x[lst_col].str.len())
     for col in x.columns.difference([lst_col])
    }).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]  

#Remove unwated data
df_perItem.drop(['address','birthdate','mail','sex','username'], axis=1, inplace=True)


# convert transaction_date strings into datetime
df_perItem['transaction_date'] = pd.to_datetime(df_perItem['transaction_date'])
df_perItem['month'] = df_perItem['transaction_date'].dt.month_name()
df_perItem['month_num'] = df_perItem['transaction_date'].dt.month
df_perItem.drop(['transaction_date'], axis=1, inplace=True)


# custom number of splits
ti_split = df_perItem['transaction_items'].str.split(',', n=3, expand=True).copy()
ti_split.rename(columns={0: 'brand', 1: 'item', 2:'quant'}, inplace=True)
ti_split['quantity'] = (ti_split['quant'].apply(lambda x: re.search(r'\d+', x).group())).astype(int)   #https://stackoverflow.com/questions/35376387/extract-int-from-string-in-pandas
ti_split.drop(['quant'], axis=1, inplace=True)

#Create a new working DF with split items
wdf = pd.concat([df_perItem,ti_split], axis=1, ignore_index=False)#Working DF
wdf.drop(['transaction_items'], axis=1, inplace=True)

#Make it look more presentable 
wdf = wdf.reindex(columns=['name', 'month', 'brand','item','quantity',])

wdf

,name,month,brand,item,quantity
0,Jennifer Campbell,January,Exotic Extras,Beef Chicharon,4
1,Melissa Kim,January,Exotic Extras,Beef Chicharon,3
2,Melissa Kim,January,HealthyKid 3+,Nutrional Milk,4
3,Melissa Kim,January,Candy City,Orange Beans,1
4,Melissa Kim,January,HealthyKid 3+,Gummy Vitamins,3
...,...,...,...,...,...
166114,Donald Andersen,June,HealthyKid 3+,Nutrional Milk,2
166115,Donald Andersen,June,Candy City,Gummy Worms,2
166116,Stephanie Russell,June,HealthyKid 3+,Nutrional Milk,1
166117,Stephanie Russell,June,Candy City,Gummy Worms,3


### Monthly sales per item

In [2]:
#Make months Categorical 
sorted_months = list(wdf['month'].unique())
wdf['month'] = pd.Categorical(wdf['month'], categories=sorted_months)

#Source for parameters: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html
sales_perMonth = wdf.pivot_table(index=["item"],values=["quantity"],columns=['month'],aggfunc={'quantity':'sum'}).copy()
sales_perMonth

quantity                                     
month               January February  March  April    May   June
item                                                            
Beef Chicharon         9665    10001   9816   9890  10028   9902
Gummy Vitamins         9681     9980  10145   9842   9948   9980
Gummy Worms            9559     9996   9986  10043   9801   9934
Kimchi and Seaweed     9676     9949   9967   9921   9773  10104
Nutrional Milk         9727     9691   9876   9786   9881   9767
Orange Beans           9774    10037   9611   9914   9964  10106
Yummy Vegetables       9959    10256   9896   9861   9735   9722

### Brand Sales Comparison

In [3]:
#Source for parameters: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html
sales_perBrand = wdf.groupby(by=["brand","item"])[['quantity']].sum().copy()
sales_perBrand

quantity
brand         item                        
Candy City    Gummy Worms            59319
              Orange Beans           59406
Exotic Extras Beef Chicharon         59302
              Kimchi and Seaweed     59390
HealthyKid 3+ Gummy Vitamins         59576
              Nutrional Milk         58728
              Yummy Vegetables       59429

### Customer Spending

In [4]:
value_perPerson = df.groupby(["name"])[['transaction_value']].sum()
value_perPerson = value_perPerson.rename(columns={'transaction_value':'total spent'})

value_perPerson

,total spent
name,
Aaron Beasley,16981
Aaron Brewer,39931
Aaron Brown,39902
Aaron Coffey,33273
Aaron Davis,46583
...,...
Zachary Valentine,45770
Zachary Ware,63767
Zachary Williams,63681


In [5]:
sales_ = wdf.pivot_table(index=["name"],values=["quantity"],columns=['month'],aggfunc={'quantity':'sum'}).copy()
sales_

quantity                              
month              January February March April May June
name                                                    
Aaron Beasley            0        0     0     9   8    1
Aaron Brewer             4        6     1     9   0   13
Aaron Brown              0        0    20    14   8    1
Aaron Coffey             1        7     9    12  12    0
Aaron Davis              6        1     0    15  15   12
...                    ...      ...   ...   ...  ..  ...
Zachary Valentine        7        6     3     2  14   19
Zachary Ware             6        9    16     5  10    8
Zachary Williams        24       23    19    14   0    0
Zachary Wilson           0        0     0     4   4   10
Zachary York             8       17     7    12  23    5

[8487 rows x 6 columns]

### Repeaters

In [13]:
#A reference for the table of repeaters
df_repRef = df_perItem.copy() 
df_repRef.drop(["transaction_items","transaction_value"], axis=1, inplace=True)
df_repRef.drop_duplicates(subset=["name", "month"], inplace=True, ignore_index=True)
df_repRef.sort_values(by=['name', 'month_num'], inplace=True, ignore_index=True)
df_repRef["Samename"] =  df_repRef.name == df_repRef.name.shift()
df_repRef["Repeaters"] =  (df_repRef.Samename & (df_repRef.month_num - 1 == df_repRef.month_num.shift()))

df_rep = df_repRef.pivot_table(values=["Repeaters"],columns=['month'],aggfunc={'Repeaters':'sum'}).copy()
df_rep = df_rep.reindex(columns = sorted_months) 

df_rep

month,January,February,March,April,May,June
Repeaters,0,5172,5216,5154,5110,5193


### Inactive

In [132]:
df_inaRef = df_repRef.copy() 
df_inaRef


,name,month,month_num,Samename,Repeaters
0,Aaron Beasley,April,4,False,False
1,Aaron Beasley,May,5,True,True
2,Aaron Beasley,June,6,True,True
3,Aaron Brewer,January,1,False,False
4,Aaron Brewer,February,2,True,True
...,...,...,...,...,...
39612,Zachary York,February,2,True,True
39613,Zachary York,March,3,True,True
39614,Zachary York,April,4,True,True
39615,Zachary York,May,5,True,True


In [133]:
df3 = df_inaRef.copy()

df3['month'] = pd.Categorical(df3['month'], categories=sorted_months)
df3 = df3.pivot_table(index=['name'],columns=['month'],aggfunc={'Repeaters':'count'}).copy()

df3

Repeaters                              
month               January February March April May June
name                                                     
Aaron Beasley             0        0     0     1   1    1
Aaron Brewer              1        1     1     1   0    1
Aaron Brown               0        0     1     1   1    1
Aaron Coffey              1        1     1     1   1    0
Aaron Davis               1        1     0     1   1    1
...                     ...      ...   ...   ...  ..  ...
Zachary Valentine         1        1     1     1   1    1
Zachary Ware              1        1     1     1   1    1
Zachary Williams          1        1     1     1   0    0
Zachary Wilson            0        0     0     1   1    1
Zachary York              1        1     1     1   1    1

[8487 rows x 6 columns]

In [185]:
df4 =df3.copy()

df4['January'] = df4.Repeaters.January == 1
df4['February'] = df4.Repeaters.February == 1
df4['March'] = df4.Repeaters.March == 1
df4['April'] = df4.Repeaters.April == 1
df4['May'] = df4.Repeaters.May == 1
df4['June'] = df4.Repeaters.June == 1

'''
df4.drop([('Repeaters','January')], axis=1, inplace=True)
df4.drop([('Repeaters','February')], axis=1, inplace=True)
df4.drop([('Repeaters','March')], axis=1, inplace=True)
df4.drop([('Repeaters','April')], axis=1, inplace=True)
df4.drop([('Repeaters','May')], axis=1, inplace=True)
df4.drop([('Repeaters','June')], axis=1, inplace=True)
'''

df5 = pd.DataFrame()


df5['January'] = df4.Repeaters.January 
df5['February'] = df4.Repeaters.February 
df5['March'] = df4.Repeaters.March 
df5['April'] = df4.Repeaters.April 
df5['May'] = df4.Repeaters.May 
df5['June'] = df4.Repeaters.June 

df5 = df5.reset_index() # New reference for inactive and engaged
df5 = 

,name,January,February,March,April,May,June
0,Aaron Beasley,0,0,0,1,1,1
1,Aaron Brewer,1,1,1,1,0,1
2,Aaron Brown,0,0,1,1,1,1
3,Aaron Coffey,1,1,1,1,1,0
4,Aaron Davis,1,1,0,1,1,1
...,...,...,...,...,...,...,...
8482,Zachary Valentine,1,1,1,1,1,1
8483,Zachary Ware,1,1,1,1,1,1
8484,Zachary Williams,1,1,1,1,0,0
8485,Zachary Wilson,0,0,0,1,1,1


In [205]:
df6 = df5.copy()

df6['1'] = False
df6['2'] = ((df6.February == 0) & (df6.January == 1))

df6

,name,January,February,March,April,May,June,1,2
0,Aaron Beasley,0,0,0,1,1,1,False,False
1,Aaron Brewer,1,1,1,1,0,1,False,False
2,Aaron Brown,0,0,1,1,1,1,False,False
3,Aaron Coffey,1,1,1,1,1,0,False,False
4,Aaron Davis,1,1,0,1,1,1,False,False
...,...,...,...,...,...,...,...,...,...
8482,Zachary Valentine,1,1,1,1,1,1,False,False
8483,Zachary Ware,1,1,1,1,1,1,False,False
8484,Zachary Williams,1,1,1,1,0,0,False,False
8485,Zachary Wilson,0,0,0,1,1,1,False,False


In [208]:
df7 = pd.DataFrame(
)

df7['January'] = df6['1']
df7['February'] = df6['2']

df7

df8 = df7.groupby(by=["brand","item"])[['quantity']].sum()

,January,February
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
8482,False,False
8483,False,False
8484,False,False
8485,False,False


In [222]:
print(df7['January'].sum())
print(df7['February'].sum())
df10 = pd.DataFrame()

df10['January'] = df7['January'].sum()

df10['February'] = df7['February'].sum()


df10.loc[len(df10.index)] = [df7['January'].sum(),df7['February'].sum()] 

df10

0
1416


,January,February
0,0,1416


### Engaged